In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader

# Kiểm tra xem có GPU không
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

# Cài đặt biến đổi dữ liệu
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),  # Chuyển ảnh từ grayscale (1 channel) sang 3 channels (RGB)
    transforms.Resize(224),  # Resize ảnh về kích thước 224x224 (để phù hợp với ResNet)
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Tải bộ dữ liệu MNIST
train_data = datasets.MNIST('.', train=True, download=True, transform=transform)
test_data = datasets.MNIST('.', train=False, download=True, transform=transform)

train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
test_loader = DataLoader(test_data, batch_size=64, shuffle=False)

# Tải mô hình ResNet đã được huấn luyện trước từ torchvision
model = models.resnet18(pretrained=True)  # Sử dụng ResNet-18 đã huấn luyện sẵn

# Chuyển đổi lớp cuối cùng (fully connected layer) của mô hình để phù hợp với số lớp của MNIST (10 class)
model.fc = nn.Linear(model.fc.in_features, 10)

# Chuyển mô hình lên GPU (nếu có GPU)
model = model.to(device)

# Đưa mô hình vào chế độ huấn luyện
model.train()

# Cài đặt hàm mất mát và tối ưu
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Huấn luyện mô hình
epochs = 5
for epoch in range(epochs):
    running_loss = 0.0
    for images, labels in train_loader:
        # Chuyển dữ liệu lên GPU
        images, labels = images.to(device), labels.to(device)
        
        # Tiến hành huấn luyện
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()

    print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss/len(train_loader)}")

# Lưu mô hình đã huấn luyện
torch.save(model.state_dict(), "mnist_resnet.pth")
